In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Conv2D, MaxPooling2D,Flatten, Dense
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import VGG19, ResNet50
from tensorflow.keras import backend as K
import matplotlib.pyplot as plt
from tensorflow.keras import layers, models

In [ ]:
# Директориї для завантаження даних
train_dir = (r'D:\KPI\7 семестр\ТМН\train')
val_dir = (r'D:\KPI\7 семестр\ТМН\val')
test_dir = (r'D:\KPI\7 семестр\ТМН\test')

# Підготовка даних
IMG_SIZE = 224
BATCH_SIZE = 32

In [ ]:
# Генератори зображень
train_datagen = ImageDataGenerator(rescale=1./255)
val_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE,
    class_mode='binary'
)

val_generator = val_datagen.flow_from_directory(
    val_dir,
    target_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE,
    class_mode='binary'
)

test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE,
    class_mode='binary'
)

In [ ]:
# Повнозв'язна модель
import tensorflow as tf
from tensorflow.keras import layers, models

fc_model = models.Sequential([
    layers.Flatten(input_shape=(IMG_SIZE, IMG_SIZE, 3)),
    layers.Dense(128, activation='relu'),
    layers.Dense(64, activation='relu'),
    layers.Dense(32, activation='relu'),
    layers.Dense(1, activation='sigmoid')
])

fc_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
fc_history = fc_model.fit(train_generator, epochs=10, validation_data=val_generator)

In [ ]:
# Згорткова нейронна мережа (CNN)
cnn_model = models.Sequential([
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=(IMG_SIZE, IMG_SIZE, 3)),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Flatten(),
    layers.Dense(64, activation='relu'),
    layers.Dense(1, activation='sigmoid')
])

cnn_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
cnn_history = cnn_model.fit(train_generator, epochs=5, validation_data=val_generator)

In [ ]:
# Перенесення навчання з використанням VGG19
vgg_base = VGG19(weights='imagenet', include_top=False, input_shape=(IMG_SIZE, IMG_SIZE, 3))
vgg_base.trainable = False  # Заморожування ваг

vgg_model = models.Sequential([
    vgg_base,
    layers.Flatten(),
    layers.Dense(64, activation='relu'),
    layers.Dense(1, activation='sigmoid')
])

vgg_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
vgg_history = vgg_model.fit(train_generator, epochs=5, validation_data=val_generator)

In [ ]:
# Перенесення навчання з використанням ResNet50
resnet_base = ResNet50(weights='imagenet', include_top=False, input_shape=(IMG_SIZE, IMG_SIZE, 3))
resnet_base.trainable = False

resnet_model = models.Sequential([
    resnet_base,
    layers.Flatten(),
    layers.Dense(64, activation='relu'),
    layers.Dense(1, activation='sigmoid')
])

resnet_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
resnet_history = resnet_model.fit(train_generator, epochs=5, validation_data=val_generator)


In [ ]:
# Порівняння продуктивності моделей
fc_test_loss, fc_test_acc = fc_model.evaluate(test_generator)
cnn_test_loss, cnn_test_acc = cnn_model.evaluate(test_generator)
vgg_test_loss, vgg_test_acc = vgg_model.evaluate(test_generator)
resnet_test_loss, resnet_test_acc = resnet_model.evaluate(test_generator)

print(f'FC Model Test Accuracy: {fc_test_acc}')
print(f'CNN Model Test Accuracy: {cnn_test_acc}')
print(f'VGG19 Model Test Accuracy: {vgg_test_acc}')
print(f'ResNet50 Model Test Accuracy: {resnet_test_acc}')

In [ ]:
# Аналіз перенавчання: Збільшення кількості епох для моделей (а) та (б)
fc_long_history = fc_model.fit(train_generator, epochs=10, validation_data=val_generator)
cnn_long_history = cnn_model.fit(train_generator, epochs=10, validation_data=val_generator)


In [ ]:
# Побудова графіків для демонстрації перенавчання
def plot_learning_curves(history, title):
    acc = history.history['accuracy']
    val_acc = history.history['val_accuracy']
    loss = history.history['loss']
    val_loss = history.history['val_loss']
    epochs = range(1, len(acc) + 1)

    plt.figure(figsize=(12, 4))
    plt.subplot(1, 2, 1)
    plt.plot(epochs, acc, label='Training Accuracy')
    plt.plot(epochs, val_acc, label='Validation Accuracy')
    plt.title(f'{title} - Training and Validation Accuracy')
    plt.legend()

    plt.subplot(1, 2, 2)
    plt.plot(epochs, loss, label='Training Loss')
    plt.plot(epochs, val_loss, label='Validation Loss')
    plt.title(f'{title} - Training and Validation Loss')
    plt.legend()
    plt.show()

plot_learning_curves(fc_long_history, 'Fully Connected Model')
plot_learning_curves(cnn_long_history, 'Convolutional Neural Network Model')